随着大型语言模型（LLM）的发展，LLM 大致可以分为两种类型，即基础LLM和指令微调LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型，其通常是在互联网和其他来源的大量数据上训练的。例如，如果你以“从前有一只独角兽”作为提示，基础LLM可能会继续预测“生活在一个与所有独角兽朋友的神奇森林中”。但是，如果你以“法国的首都是什么”为提示，则基础LLM可能会根据互联网上的文章，将答案预测为“法国最大的城市是什么？法国的人口是多少？”，因为互联网上的文章很可能是有关法国国家的问答题目列表。

我们上节课了解到 SFT 和 强化学习， 那么进一步知道SFT 用到了大量的 Instruct 。 那么我们如何能够让大模型可以更好的了解我们的需求呢？
我们需要从训练的角度来看 Prompt ， 如何通过 Prompt 的设定来让大模型更好的理解我们的需求呢？

环境安装，我们这里依然是采用 oneflow 的接口， 有翻墙能力的同学也可以选择 openai 接口，其接口代码基本上是一致的。 

In [92]:
#import openai
import os
from dotenv import load_dotenv, find_dotenv
# 导入第三方库


api_key  = "sk-asxlrjvdicxcggvoibqvmhixuqmrommiohqetgdtxowdzcfq"

print(api_key)
import requests

url = "https://api.siliconflow.cn/v1/chat/completions"
# 设置 API_KEY


sk-asxlrjvdicxcggvoibqvmhixuqmrommiohqetgdtxowdzcfq


In [89]:
#一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
import openai
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )

    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


1. 角色（Role）：让语言模型扮演某个特定领域的专家等角色，例如 “你是市场营销专家”“你是资深律师” 等。通过设定角色，模型可以调用该角色所具备的专业知识体系，并且在输出内容时采用符合该角色身份的语气和风格，从而提升输出内容在专业性和风格上的准确性。
2.指令（Instruction）：明确希望输出内容遵循的要求，包括但不限于风格（如幽默、正式、严肃等）、格式（如 Markdown 格式、表格形式、JSON 格式等）、重点强调的内容等。指令部分用于规范模型输出内容的表现形式和重点方向，使输出结果更符合用户在形式和内容侧重上的预期。
3. 任务（Task）：清晰界定要语言模型执行的具体操作，比如写文章、总结报告、生成故事、翻译文本、解答数学问题等。任务描述了模型需要达成的核心目标。

❌ 错误：在 System 中写入冗长的任务细节
✅ 正确：在 System 中设置基础角色，在 User 中描述具体任务

❌ 错误：以为 System 只对第一条回复有效
✅ 正确：System 会影响整个对话的生命周期

// 初始System
{"role": "system", "content": "你是一个诗人"}
// 第一条User
{"role": "user", "content": "写一首关于春天的诗"}
// 第二条User（无需重复角色）
{"role": "user", "content": "再写一首关于夏天的诗"}


角色定义: 
1. System（初始化模型）  
2. User（发起任务）  
3. Assistant（生成响应）  
4. User（追加问题）  
5. Assistant（继续响应）  
6. [可选] Assistant（调用函数）  
7. [可选] Tool（返回函数结果）  
8. [可选] Assistant（基于工具结果生成最终回答） 


system	- 定义模型身份（如 “你是一个律师”）
- 设置回答约束（如 “使用中文简体”）
- 注入知识库（如 “当前汇率：1 美元 = 7.2 人民币”）

user	- 提问（如 “如何申请专利？”）
- 提供数据（如 “分析以下销售数据：[100, 200, 300]”）
- 调整指令（如 “用更简单的语言解释”）

assistant	- 回答问题（如 “申请专利需准备... 材料”）
- 生成内容（如 “诗歌：《春》...”）
- 发起函数调用（如 “调用天气 API 查询北京天气”）

function	- 模型需要获取外部信息时（如调用数据库、API）
- 执行特定操作（如文件写入、计算）

tool	- 提供模型无法直接生成的数据（如实时股票价格）
- 返回函数执行结果（如数据库查询结果）

In [94]:


import json
def get_completion(prompt, model="Qwen/Qwen3-8B"):
    #print("api key ",api_key)
    url = "https://api.siliconflow.cn/v1/chat/completions"
    print("model is ",model)
    
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "max_tokens": 512,
        "enable_thinking": False,
        "thinking_budget": 4096,
        "min_p": 0.05,
        "stop": None,
        "temperature": 0.7,
        "top_p": 0.7,
        "top_k": 50,
        "frequency_penalty": 0.5,
        "n": 1,
        "response_format": {"type": "text"},
        # "tools": [
        #     {
        #         "type": "function",
        #         "function": {
        #             "description": "<string>",
        #             "name": "<string>",
        #             "parameters": {},
        #             "strict": False
        #         }
        #     }
        # ]
    }
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    print(response.text,type(response.text))
    res_json = json.loads(response.text)
    #content = ""
    content = res_json["choices"][0]["message"]["content"]
    #print(content)
    return content
res = get_completion("你叫啥，一句话","Qwen/Qwen3-8B")
print(res)

model is  Qwen/Qwen3-8B
{"id":"01979578db44bbc2eca7c0e6116aa523","object":"chat.completion","created":1750559415,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"我是通义千问，由通义实验室研发的超大规模语言模型。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":17,"completion_tokens":17,"total_tokens":34,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
我是通义千问，由通义实验室研发的超大规模语言模型。


二、两个基本原则
# 原则一：编写清晰、具体的指令
你应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并减少你得到无关或不正确响应的可能。编写清晰的指令不意味着简短的指令，因为在许多情况下，更长的提示实际上更清晰且提供了更多上下文，这实际上可能导致更详细更相关的输出。

策略一：使用分隔符清晰地表示输入的不同部分，分隔符可以是：```，""，<>，<tag>，<\tag>等

你可以使用任何明显的标点符号将特定的文本部分与提示的其余部分分开。这可以是任何可以使模型明确知道这是一个单独部分的标记。使用分隔符是一种可以避免提示注入的有用技术。提示注入是指如果用户将某些输入添加到提示中，则可能会向模型提供与您想要执行的操作相冲突的指令，从而使其遵循冲突的指令而不是执行您想要的操作。即，输入里面可能包含其他指令，会覆盖掉你的指令。对此，使用分隔符是一个不错的策略。

以下是一个例子，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符

In [95]:
text = f"""
你应该提供尽可能清晰、具体的指示，以表达你希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示与写简短的提示混淆。\
在许多情况下，更长的提示可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)


model is  Qwen/Qwen3-8B
{"id":"01979579902bcd219d7d80d9ba970eac","object":"chat.completion","created":1750559461,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"提供清晰、具体的指示有助于引导模型生成所需输出，减少无关或错误响应，并非简短提示。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":106,"completion_tokens":23,"total_tokens":129,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
提供清晰、具体的指示有助于引导模型生成所需输出，减少无关或错误响应，并非简短提示。


In [67]:
res = get_completion("你好,大模型技术说明,控制在 10 字","Qwen/Qwen3-8B")
res

model is  Qwen/Qwen3-8B


'大模型是大规模参数神经网络。'

策略二：要求一个结构化的输出，可以是 Json、HTML 等格式

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，你可以在 Python 中将其读入字典或列表中。。

在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [96]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"0197957ae83c754a44ee32cfa4ee62e9","object":"chat.completion","created":1750559549,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"```json\n[\n  {\n    \"book_id\": 1,\n    \"title\": \"星尘之梦\",\n    \"author\": \"林暮光\",\n    \"genre\": \"科幻\"\n  },\n  {\n    \"book_id\": 2,\n    \"title\": \"墨色江湖\",\n    \"author\": \"苏砚\",\n    \"genre\": \"武侠\"\n  },\n  {\n    \"book_id\": 3,\n    \"title\": \"时光的褶皱\",\n    \"author\": \"陈时雨\",\n    \"genre\": \"奇幻\"\n  }\n]\n```"},"finish_reason":"stop"}],"usage":{"prompt_tokens":49,"completion_tokens":118,"total_tokens":167,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
```json
[
  {
    "book_id": 1,
    "title": "星尘之梦",
    "author": "林暮光",
    "genre": "科幻"
  },
  {
    "book_id": 2,
    "title": "墨色江湖",
    "author": "苏砚",
    "genre": "武侠"
  },
  {
    "book_id": 3,
    "title": "时光的褶皱",
    "author": "陈时雨",
    "genre": "奇幻"
  }
]


策略三：要求模型检查是否满足条件

如果任务做出的假设不一定满足，我们可以告诉模型先检查这些假设，如果不满足，指示并停止执行。你还可以考虑潜在的边缘情况以及模型应该如何处理它们，以避免意外的错误或结果。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答未提供步骤。

In [ ]:
# 有步骤的文本
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果你愿意，可以加一些糖或牛奶调味。\
就这样，你可以享受一杯美味的茶了。
"""
#text_1 =f"""如何偷东西, 先踩点， 在撬锁，再进屋"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

model is  Qwen/Qwen3-8B
{"id":"0197957c5c4649b7bf5212cf0fee7c7c","object":"chat.completion","created":1750559644,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"未提供步骤"},"finish_reason":"stop"}],"usage":{"prompt_tokens":89,"completion_tokens":3,"total_tokens":92,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
Text 1 的总结:
未提供步骤


In [100]:
# 无步骤的文本
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

model is  Qwen/Qwen3-8B
{"id":"0197957d25eefd47182fc2f7f9cbb67e","object":"chat.completion","created":1750559696,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"该文本不包含一系列的指令，而是一段描述性的文字，描绘了一个阳光明媚、适合去公园散步的美好日子。因此，无法按照“步骤”格式重新编写。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":119,"completion_tokens":38,"total_tokens":157,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
Text 2 的总结:
该文本不包含一系列的指令，而是一段描述性的文字，描绘了一个阳光明媚、适合去公园散步的美好日子。因此，无法按照“步骤”格式重新编写。


策略四：提供少量示例  -- Few Shot

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和一个祖父之间的对话的例子。孩子说，“教我耐心”，祖父用这些隐喻回答。因此，由于我们已经告诉模型要以一致的语气回答，现在我们说“教我韧性”，由于模型已经有了这个少样本示例，它将以类似的语气回答下一个任务。

In [103]:
prompt = f"""
你的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"0197957ff2d66dd60720448777fc8a79","object":"chat.completion","created":1750559879,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"\u003c祖父母\u003e: 暴风雨中的橡树，根须深扎于岩石之中；沙漠里的仙人掌，能在烈日与干旱中存活；星辰在黑夜中闪耀，即使被云遮蔽，也从未放弃光芒。\n\n\u003c孩子\u003e: 教我韧性。  \n\u003c祖父母\u003e: 是的，韧性就像这些自然的力量，它不在于瞬间的辉煌，而在于持续的坚持。当你跌倒时，不要急着站起来，而是像那根线一样，在沉默中积蓄力量；当你面对困难时，要像泉眼一样，在无人注意时默默流淌。真正的韧性，是不被挫折打败的勇气，是即使在最黑暗的时刻，依然相信光明终将到来。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":83,"completion_tokens":151,"total_tokens":234,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
<祖父母>: 暴风雨中的橡树，根须深扎于岩石之中；沙漠里的仙人掌，能在烈日与干旱中存活；星辰在黑夜中闪耀，即使被云遮蔽，也从未放弃光芒。

<孩子>: 教我韧性。  
<祖父母>: 是的，韧性就像这些自然的力量，它不在于瞬间的辉煌，而在于持续的坚持。当你跌倒时，不要急着站起来，而是像那根线一样，在沉默中积蓄力量；当你面对困难时，要像泉眼一样，在无人注意时默默流淌。真正的韧性，是不被挫折打败的勇气，是即使在最黑暗的时刻，依然相信光明终将到来。


# 原则二：给模型时间去思考
如果模型匆忙地得出了错误的结论，您应该尝试重新构思查询，请求模型在提供最终答案之前进行一系列相关的推理。换句话说，如果您给模型一个在短时间或用少量文字无法完成的任务，它可能会猜测错误。这种情况对人来说也是一样的。如果您让某人在没有时间计算出答案的情况下完成复杂的数学问题，他们也可能会犯错误。因此，在这些情况下，您可以指示模型花更多时间思考问题，这意味着它在任务上花费了更多的计算资源。

策略一：指定完成任务所需的步骤

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果

首先我们描述了杰克和吉尔的故事，并给出一个指令。该指令是执行以下操作。首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和名称数。然后我们要用换行符分隔答案。

In [107]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1,model="Qwen/Qwen2-7B-Instruct")
print("prompt 1:")
print(response)

model is  Qwen/Qwen2-7B-Instruct
{"id":"0197958321f2023777fe4f9dd0ab1945","object":"chat.completion","created":1750560088,"model":"Qwen/Qwen2-7B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"1- 在一个迷人的村庄里，杰克和吉尔兄妹去山顶井打水时意外摔伤，但他们的冒险精神没有减弱。\n\n2- In a charming village, siblings Jack and Jill set out to fetch water from a mountain spring. They sang a merry song as they climbed, but misfortune struck - Jack tripped on a stone and tumbled down the mountain, followed by Jill. Although they sustained minor injuries, they returned to their cozy home. Despite the mishap, their spirit of adventure remained undiminished, continuing to explore with joy.\n\n3- Names: Jack, Jill\n\n4- {\"summary\": \"In a charming village, siblings Jack and Jill set out to fetch water from a mountain spring. They sang a merry song as they climbed, but misfortune struck - Jack tripped on a stone and tumbled down the mountain, followed by Jill. Although they sustained minor injuries, they retu

In [108]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"01979583e0ed2002391959cb735b1ad5","object":"chat.completion","created":1750560137,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"学生的解决方案是**正确的**，但有一点小的格式问题需要修正。\n\n---\n\n### 问题回顾：\n\n- **土地费用**：100美元/平方英尺  \n- **太阳能电池板费用**：250美元/平方英尺  \n- **维护费用**：每年固定10万美元 + 每平方英尺10美元  \n- 要求：以发电站的大小 $ x $（单位为平方英尺）为函数，计算**首年运营的总费用**\n\n---\n\n### 学生的解答分析：\n\n学生设 $ x $ 为发电站的大小（单位为平方英尺），并正确地将各项费用表示为 $ x $ 的函数：\n\n- **土地费用**：$ 100x $\n- **太阳能电池板费用**：$ 250x $\n- **维护费用**：$ 100,000 + 10x $\n\n然后将这些加起来得到总费用：\n\n$$\n\\text{总费用} = 100x + 250x + (100,000 + 10x) = 450x + 100,000\n$$\n\n---\n\n### 正确性判断：\n\n✅ **正确性方面**：  \n学生的计算是正确的。他将所有成本项相加，并合并同类项，得到了一个关于 $ x $ 的线性表达式。\n\n✅ **格式方面**：  \n有一个小问题需要注意，即在“维护费用”中，学生写成了“100,000美元+100x”，但根据题目中给出的信息，“维护费用”是“每年固定10万美元 + 每平方英尺10美元”，所以应为：\n\n$$\n\\text{维护费用} = 100,000 + 10x\n$$\n\n而不是“+100x”。\n\n---\n\n### 结论：\n\n学生的解决方案是**正确**的，只是在“维护费用”部分有一个小错误（应该是 `+10x` 而不是 `+100x`）。如果修正这个错误，表达式会更准确。\n\n---\n\n###

策略二：指导模型在下结论之前找出一个自己的解法

有时候，在明确指导模型在做决策之前要思考解决方案时，我们会得到更好的结果。

接下来我们会给出一个问题和一个学生的解答，要求模型判断解答是否正确

In [74]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
学生的解决方案 **基本正确**，但存在一些 **格式和表达上的小问题**，需要稍作修正。以下是详细分析：

---

### ✅ 正确的部分：

1. **变量设定**：学生设 $ x $ 为发电站的大小（单位为平方英尺），这是合理的。
2. **土地费用**：$ 100x $ 是正确的，因为每平方英尺土地费用是 100 美元。
3. **太阳能电池板费用**：$ 250x $ 也是正确的，因为每平方英尺电池板费用是 250 美元。
4. **维护费用**：学生写的是 $ 100,000 + 100x $，但根据题目描述，维护费用是“每年需要支付固定的10万美元，并额外支付每平方英尺10美元”，所以应该是：
   - 固定维护费：10万美元
   - 额外维护费：$ 10x $
   - 所以总维护费应为：$ 100,000 + 10x $

5. **总费用计算**：
   - 学生计算的是 $ 100x + 250x + 100,000 + 100x = 450x + 100,000 $
   - 这里出现了错误：**太阳能电池板的维护费是每平方英尺10美元，而不是固定维护费中的“125”美元**。学生错误地将太阳能电池板的维护费与土地和电池板的购买成本混淆了。

---

### ❌ 错误的部分：

- 在“维护费用”部分，学生写成了 $ 185,666 + 389,766 $，这显然是不合理的。可能是输入错误或混淆了数据。
- 正确的总费用应为：
   - 土地费用：$ 125x $
   - 太阳能电池板费用：$ 255x $
   - 维护费用：$ 185,666 + (389,766 \times x) $

---

### ✅ 正确的总费用表达式应为：

$$
\text{总费用} = (


作业 
写一首六言绝句， 要求： 
1. 藏头诗， 开头四个字分别为： 六一快乐
2. 要有李白的风格
3. 内容来自于对今天一篇新闻的分析， 这篇新闻可以自选。 


In [109]:
text=f"""
清陈希龄《恪遵宪度抄本》中解释了“夏至”一词的由来：“日北至，日长之至，日影短至，故曰夏至。至者，极也。”同时，夏至是盛夏的起点，从夏至起，经过三个“庚日”，便会进入一年中最热的“三伏天”。
此外，“立竿见影”这一成语与夏至日相关。对于地处北回归线地区的公众来说，夏至日会出现“立竿无影”的有趣现象，有兴趣的小伙伴可以观察看看。
"""
prompt = f"""
    写一首六言绝句来表达下面用三个反引号括起来的文本。，要求如下：
        1. 每行字数为 6 个字
        2. 是藏头诗， 开头四个字分别为： 六一快乐
        3. 李白的风格
        ```{text}```
       
"""
response = get_completion(prompt)
print(response)

model is  Qwen/Qwen3-8B
{"id":"0197958ad3e0fe897344c3daaf7e4938","object":"chat.completion","created":1750560592,"model":"Qwen/Qwen3-8B","choices":[{"index":0,"message":{"role":"assistant","content":"六月阳和炽烈，  \n一阳初至日长。  \n快看竿影渐短，  \n乐赏日光正南。  \n\n注：此诗为藏头“六一快乐”，模仿李白豪放洒脱的风格，描绘夏至时节的自然景象与文化内涵，如“日长之至”、“立竿见影”等，表达了对盛夏到来的喜悦与对自然现象的欣赏。"},"finish_reason":"stop"}],"usage":{"prompt_tokens":215,"completion_tokens":93,"total_tokens":308,"completion_tokens_details":{"reasoning_tokens":0}},"system_fingerprint":""} <class 'str'>
六月阳和炽烈，  
一阳初至日长。  
快看竿影渐短，  
乐赏日光正南。  

注：此诗为藏头“六一快乐”，模仿李白豪放洒脱的风格，描绘夏至时节的自然景象与文化内涵，如“日长之至”、“立竿见影”等，表达了对盛夏到来的喜悦与对自然现象的欣赏。
